In [2]:
import pandas as pd
import numpy as np
from scipy.integrate import quad
import itertools

# Constant market (CRRA and Habit)

In [131]:
# Fixed parameters
t = 0
W = 1000
delta = 0.02

_lambda = 0.3
sigma = 0.2
r = 0.04

# Functions
def F(t, T, r, alpha, beta): # Needed for habit utility
    return (1 - np.exp(-(r + beta - alpha)*(T - t))) / (r + beta - alpha) 

def compute_allocation(h, gamma, T, alpha, beta): # Computes optimal allocation in the risky assets - in general form works for habit. Choose alpha, beta and h = 0 for CRRA
    f_val = F(t, T, r, alpha, beta)
    aktieandel = (_lambda / (gamma * sigma)) * (W - h * f_val) / W
    return aktieandel

# Parameter ranges
hs = [0]
gammas = [5]
Ts = [5]
alphas = [0]
betas = [0]

# Simulation
results = []

for h, gamma, T, alpha, beta in itertools.product(hs, gammas, Ts, alphas, betas):
    aktieandel = compute_allocation(h, gamma, T, alpha, beta)
    cash_position = 1 - aktieandel
    results.append({
        'h': h,
        'gamma': gamma,
        'T': T,
        'alpha': alpha,
        'beta': beta,
        'stock_weight_%': round(aktieandel * 100, 2),
        'cash_weight_%': round(cash_position * 100, 2),
        'stock/cash ratio': aktieandel/cash_position
    })

# Output
df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

 h  gamma  T  alpha  beta  stock_weight_%  cash_weight_%  stock/cash ratio
 0      5  5      0     0            30.0           70.0          0.428571


# Stochastic market (CRRA and Habit)

In [3]:
# Parameters
t = 0 
W = 1000
delta = 0.02 
kappa = 0.3

gamma = 5

T = 10 
r = 0.04

Lambda = 0.3
Lambda_cap = 0.3

Sigma = 0.2 #sigma lambda
sigma = 0.2

h = 5
alpha = 0
beta = 0

# Auxiliary functions
b = kappa - (1 - 1/gamma) * Sigma
theta = np.sqrt(b**2 - (Sigma**2 * (1 - gamma)) / gamma**2)

def A0(t, T):
    term1 = -(delta / gamma) * (T - t)
    term2 = -r * (1 - 1/gamma) * (T - t)
    term3 = -0.5 * np.log((2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t)))) / (2 * theta))
    term4 = ((1 - gamma) / (2 * gamma**2)) * (
        ((kappa**2 * Lambda_cap**2) / theta**2 + Sigma**2 / (theta + b)) * (T - t) +
        ((kappa**2 * Lambda_cap**2) / theta**3) *
        ((theta - 2 * b) * np.exp(-2 * theta * (T - t)) + 4 * b * np.exp(-theta * (T - t)) - theta - 2 * b) /
        (2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t))))
    )
    return term1 + term2 + term3 + term4

def A1(t, T):
    return ((1 - gamma) / gamma**2) * (kappa * Lambda_cap / theta) * ((1 - np.exp(-theta * (T - t)))**2) / \
           (2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t))))

def A2(t, T):
    return ((1 - gamma) / gamma**2) * (1 - np.exp(-2 * theta * (T - t))) / \
           (2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t))))

def F(t, T):
    return (1 - np.exp(-(r + beta - alpha) * (T - t))) / (r + beta - alpha)

def numerator_D1(t, T):
    integrand = lambda s: (A1(t, s) + A2(t, s) * Lambda) * (1 + alpha * F(s, T))**(1 - 1/gamma) * \
               np.exp(A0(t, s) + A1(t, s) * Lambda + 0.5 * A2(t, s) * Lambda**2)
    result, _ = quad(integrand, t, T, limit=200)
    return result

def denominator_D1(t, T):
    integrand = lambda s: (1 + alpha * F(s, T))**(1 - 1/gamma) * \
               np.exp(A0(t, s) + A1(t, s) * Lambda + 0.5 * A2(t, s) * Lambda**2)
    result, _ = quad(integrand, t, T, limit=200)
    return result

def D1(t, T):
    return numerator_D1(t, T) / denominator_D1(t, T)

# Safe ratio calculation function
def safe_divide(numerator, denominator):
    if np.isclose(denominator, 0):
        return 0
    else:
        return numerator / denominator

# Portfolio weights
my = Lambda / (gamma * sigma) * (W - h * F(t, T)) / W
hed = -(Sigma / sigma) * D1(t, T) * (W - h * F(t, T)) / W

# Total stock weight
total_stock_weight = my + hed

# Cash weight
cash_weight = 1 - total_stock_weight

# Safe Ratios
myopic_hedging_ratio = safe_divide(hed, my)

# Print results
print(f"Myopic weight (%): {my*100:.1f}")
print(f"Hedging weight (%): {hed*100:.1f}")
print(f"Total stock weight (%): {total_stock_weight*100:.1f}")
print(f"Cash weight (%): {cash_weight*100:.1f}")
print(f"Hedging-to-myopic ratio: {myopic_hedging_ratio:.2f}")


Myopic weight (%): 28.8
Hedging weight (%): 17.5
Total stock weight (%): 46.3
Cash weight (%): 53.7
Hedging-to-myopic ratio: 0.61
